## Parsing the CSSE Daily Data for COVID 19:
https://github.com/CSSEGISandData/COVID-19

Using the code provided by Prof. James Sharpnack
https://github.com/jsharpna/CovidResponse208

The population dataset is from the 2019 census

and 
## Parsing the social Distancing data from Safegraph


In [71]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotnine as p9
import mizani
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import ParameterGrid

plt.style.use('ggplot')

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, losses

from scipy.stats import nbinom

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))
tf.keras.backend.set_floatx('float64')


TensorFlow version: 2.1.0
Eager execution: True


In [2]:
X_tr = pd.read_csv('../data/X_tr.csv')   
y_tr = pd.read_csv('../data/y_tr.csv') 
print(X_tr.shape)
print(y_tr.shape)

(3573, 4)
(3573, 1)


In [3]:
print([y_tr['Incident_Rate'] < 0.5].shape)
X_tr = X_tr[y_tr['Incident_Rate'] >= 0.5]
y_tr = y_tr[y_tr['Incident_Rate'] >= 0.5]

In [32]:
X_tr.head()

,Province_State,Population,Elapsed_Days,Percentage_Home
157,Nebraska,1934408.0,0,19.236149
265,Washington,7614893.0,0,23.675703
282,Washington,7614893.0,1,24.347069
307,Washington,7614893.0,2,24.577092
337,Washington,7614893.0,3,27.524526


In [33]:
X_tr.describe()

,Population,Elapsed_Days,Percentage_Home
count,2.920000e+03,2920.000000,2920.000000
mean,6.420689e+06,27.018493,37.653490
std,7.291724e+06,16.636420,6.623412
min,5.787590e+05,0.000000,17.885169
25%,1.787065e+06,13.000000,32.934542
50%,4.467673e+06,27.000000,37.216028
75%,7.614893e+06,41.000000,41.818998
max,3.951222e+07,66.000000,69.594465


In [34]:
X_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2920 entries, 157 to 3572
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Province_State   2920 non-null   object 
 1   Population       2920 non-null   float64
 2   Elapsed_Days     2920 non-null   int64  
 3   Percentage_Home  2920 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 114.1+ KB


In [21]:
#standardizer function which standardizes the numerical data and encodes the categorical data with LabelEncoder
def standardize1 (X_tr):
    #standadizing the numercial values
    scaler = StandardScaler()
    X = X_tr.values[:,1:4]
    scaler.fit(X)
    X_tr_std_num = np.asarray(scaler.transform(X)).astype(np.float64)

    #standadizing the categorical values
    encoder = LabelEncoder()
    X = X_tr.values[:,0].reshape(-1, 1)
    encoder.fit(X)
    X_tr_std_cat =  encoder.transform(X).reshape(-1, 1)

    return np.concatenate((X_tr_std_cat, X_tr_std_num),axis=1)

X_tr_std1 = standardize1(X_tr)

(2920, 3)
[[27.         -0.6153619  -1.62433509 -2.78111875]
 [48.          0.16380336 -1.62433509 -2.11072188]
 [48.          0.16380336 -1.56421571 -2.00934203]
 ...
 [49.         -0.63487517  1.26139521 -0.78225105]
 [50.         -0.08205976  1.68223088 -0.54472124]
 [51.         -0.80130986  1.68223088 -0.64127019]]


In [35]:
#standardizer function which standardizes the numerical data and encodes the categorical data with one hot encoding
def standardize2 (X_tr):
    #standadizing the numercial values
    scaler = StandardScaler()
    X = X_tr.values[:,1:4]
    scaler.fit(X)
    X_tr_std_num = np.asarray(scaler.transform(X)).astype(np.float64)

    #standadizing the categorical values
    encoder = OneHotEncoder(sparse=False)
    X = X_tr.values[:,0].reshape(-1, 1)
    encoder.fit(X)
    X_tr_std_cat =  encoder.transform(X)

    return np.concatenate((X_tr_std_cat, X_tr_std_num),axis=1)

X_tr_std2 = standardize2(X_tr)

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -0.6153619  -1.62433509
  -2.78111875]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.

In [45]:
X_tr_std = X_tr_std1
print(X_tr_std.shape)
print(X_tr_std[:5,:])

(2920, 4)
[[27.         -0.6153619  -1.62433509 -2.78111875]
 [48.          0.16380336 -1.62433509 -2.11072188]
 [48.          0.16380336 -1.56421571 -2.00934203]
 [48.          0.16380336 -1.50409633 -1.97460725]
 [48.          0.16380336 -1.44397695 -1.5295287 ]]


In [39]:
y_tr.head()

,Incident_Rate
157,0.568649
265,0.538419
282,0.945516
307,1.116234
337,1.444538


In [40]:
y_tr.describe()

,Incident_Rate
count,2920.000000
mean,139.077962
std,232.835254
min,0.503619
25%,16.006792
50%,60.191737
75%,145.736850
max,1712.395998


In [41]:
y_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2920 entries, 157 to 3572
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Incident_Rate  2920 non-null   float64
dtypes: float64(1)
memory usage: 45.6 KB


In [78]:
X_te = pd.read_csv('../data/X_te.csv')   
y_te = pd.read_csv('../data/y_te.csv') 
print(X_te.shape)
print(y_te.shape)

(780, 4)
(780, 1)


In [82]:
X_te = X_te[y_te['Incident_Rate'] >= 0.5]
y_te = y_te[y_te['Incident_Rate'] >= 0.5]

X_te_std1 = standardize1(X_te)
X_te_std2 = standardize2(X_te)

(780, 3)


In [83]:
X_te_std = X_te_std1
print(X_te_std.shape)
print(X_te_std[:5,:])

(780, 4)
[[ 0.         -0.20335228 -0.6739714  -1.28501501]
 [ 1.         -0.78022736 -0.87141654 -0.12025688]
 [ 2.          0.12514807 -0.97013911  0.66406584]
 [ 3.         -0.46407212 -0.57524883 -1.27191632]
 [ 4.          4.58255718 -0.27908111  1.09397147]]


In [84]:
y_te.describe()

,Incident_Rate
count,780.000000
mean,402.180122
std,390.947895
min,42.852678
25%,160.799613
50%,246.760960
75%,478.752059
max,1858.348711


## Designing our regression models

without any delays

1- Random Forests Regression

2- Support Vector Machine Regression 

3- ANN Regression based on the negative binomial loss function and Adam Optimizer

In [109]:
X = X_tr_std
y = y_tr.values

Xt = X_te_std
yt = y_te.values

X = np.asarray(X).astype(np.float64)
y = np.asarray(y).astype(np.float64)
Xt = np.asarray(Xt).astype(np.float64)
yt = np.asarray(yt).astype(np.float64)

# Random Forest Regressor

In [89]:
params = {
    "n_estimators": [10, 50, 100, 200],
    "max_depth": [4, 8, 12, 16, 20, 32, 64]
}
best_score = -np.Inf 
rf = RandomForestRegressor()
for g in ParameterGrid(params):
    rf.set_params(**g)
    rf.fit(X, y)
    # save if best
    score = rf.score(Xt, yt)
    if score > best_score:
        best_score = score
        best_grid = g

print("r2 score: %0.5f" % best_score )
print("Grid:", best_grid)


OOB: 0.09723
Grid: {'max_depth': 16, 'n_estimators': 10}


# Support Vector Machine Regressor

In [92]:
params = {
    "C" : [0.01, 0.1, 0.2, 0.5, 1, 2, 5],
    "kernel" : ["rbf","poly","sigmoid", "linear"],
    "gamma" : ["scale"]
}

best_score = -np.Inf 
svm = SVR()
for g in ParameterGrid(params):
    svm.set_params(**g)
    svm.fit(X, y)
    # save if best
    score = svm.score(Xt, yt)
    if score > best_score:
        best_score = score
        best_grid = g

print("r2 score: %0.5f" % best_score )
print("Grid:", best_grid)

r2 score: -0.70803
Grid: {'C': 5, 'gamma': 'scale', 'kernel': 'linear'}


# Artificial Neural Net Regressor

In [110]:
train_dataset = tf.data.Dataset.from_tensor_slices((X, y))
test_dataset = tf.data.Dataset.from_tensor_slices((Xt, yt))

batch_size = 32

#no shuffle
# train_dataset = train_dataset.shuffle(200)
train_dataset = train_dataset.batch(batch_size)
X,y = next(iter(train_dataset))
X

<tf.Tensor: shape=(32, 4), dtype=float64, numpy=
array([[27.        , -0.6153619 , -1.62433509, -2.78111875],
       [48.        ,  0.16380336, -1.62433509, -2.11072188],
       [48.        ,  0.16380336, -1.56421571, -2.00934203],
       [48.        ,  0.16380336, -1.50409633, -1.97460725],
       [48.        ,  0.16380336, -1.44397695, -1.5295287 ],
       [32.        ,  1.78765747, -1.62433509, -1.36014191],
       [48.        ,  0.16380336, -1.38385757, -0.8831    ],
       [32.        ,  1.78765747, -1.56421571, -2.33151197],
       [48.        ,  0.16380336, -1.32373819, -1.97009169],
       [ 8.        , -0.78389125, -1.62433509, -1.84484238],
       [21.        ,  0.06471654, -1.62433509, -2.30655771],
       [32.        ,  1.78765747, -1.50409633, -2.09568061],
       [48.        ,  0.16380336, -1.2636188 , -1.93146949],
       [ 5.        , -0.09079692, -1.62433509, -2.2611713 ],
       [ 8.        , -0.78389125, -1.62433509, -1.46458267],
       [21.        ,  0.06471654, -1

# skip

In [98]:
y_true = np.random.randint(0, 2, size=(2, 3))
print(y_true)
y_pred = np.random.random(size=(2, 3))
print(y_pred)
loss = tf.keras.losses.poisson(y_true, y_pred)
assert loss.shape == (2,)
y_pred = y_pred + 1e-7
assert np.allclose(
    loss.numpy(), np.mean(y_pred - y_true * np.log(y_pred), axis=-1),
    atol=1e-5)

[[1 0 1]
 [1 1 0]]
[[0.74312899 0.90220121 0.02931784]
 [0.52646074 0.89741392 0.87562621]]


In [102]:
## Create the losses the poisson loss Asuumes Mean = Variance
loss_func = losses.Poisson()

# logistic_loss = losses.BinaryCrossentropy(from_logits=True)

In [103]:
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_pred = model(x, training= training)

  return loss_func(y_true= y, y_pred= y_pred)

In [104]:
## Gradient tape lets TF know with respect to what to take gradients inputs = X an targets = Y
def grad(model, inputs, targets):
  with tf.GradientTape() as tape: # getting gradient tapes for the automatix differentiation
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [113]:
# the first model with relu activation function and 3 units in the hidden layer
model = Sequential([
  layers.Dense(30, input_dim = 4, activation="relu"), 
  layers.Dense(1)
])

y_pred = model(X)
print(model.summary())

print(loss(model, X, y, training= False))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 30)                150       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 31        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________
None
tf.Tensor(nan, shape=(), dtype=float64)


In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')

# test

In [99]:
import tensorflow as tf
import tensorflow as tf
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

def gen_data(N = 10000):
    data = np.random.uniform(-1, 1, (N, 3))
    data = sm.add_constant(data)
    data = pd.DataFrame(data, columns = ['intercept', 'Var1', 'Var2', 'Var3'])
    lam = np.exp(-2*data['intercept'] + data['Var1'] - 0.5*data['Var2'] + 0.3*data['Var3'] )
    resp = np.random.poisson(lam = lam)
    data['lam'] = lam
    data['response'] = resp
    return data

dtrain = gen_data()
dtrain.drop('lam', axis=1)

X = tf.constant(dtrain[['intercept', 'Var1', 'Var2', 'Var3']].values, name = 'X', dtype=tf.float32)
# <tf.Tensor 'X:0' shape=(10000, 4) dtype=float32>

y = tf.constant(value = list(dtrain['response']), dtype = tf.float32, name='y', shape=(dtrain.shape[0], 1))
# <tf.Tensor 'y:0' shape=(10000, 1) dtype=float32>

parameters = tf.Variable(tf.zeros([4, 1])) #Initial Values
# <tf.Variable 'Variable:0' shape=(4, 1) dtype=float32_ref>

logits = tf.matmul(X, parameters, name="logits")
# <tf.Tensor 'logits:0' shape=(10000, 1) dtype=float32> 

y_hat = tf.exp(logits)

# create the loss
loss = tf.reduce_mean(-y*tf.log(y_hat)+y_hat)
# last term can be avoided since it doesn't depend on y_pred
# however keeping it gives a nice lower bound to zero
# tf.lgamma computes the log of the absolute value of Gamma(x) element-wise

learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

n_epochs = 10000

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        sess.run(training_op)
        if epoch % 100 == 0:
            print("Epoch", epoch, "Loss = ", loss.eval())
    best_parameters = parameters.eval()
    
print(best_parameters)

AttributeError: module 'tensorflow' has no attribute 'log'